In [1]:
import requests
import json
from tqdm.auto import tqdm
from collections import defaultdict
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta
from time import sleep

In [69]:

# Собираю json

from time import time


period = 7 #можно задать любой до 30 дней


t = datetime.datetime.now() 
delta = timedelta(hours=0.5) 
t1 = t - delta
sber = '3529'
page = 1
per_page = 100




all_vacancy_ids = []
for i in tqdm(range(period*96)):

    url = f'https://api.hh.ru/vacancies?employer_id={sber}&date_to={t.isoformat()}&date_from={t1.isoformat()}'
    res = requests.get(url)
    vacancies = res.json()
    vacancy_ids = [el.get('id') for el in vacancies.get('items')]
    
    all_vacancy_ids.extend(vacancy_ids)
    if len(all_vacancy_ids) !=0 and len(all_vacancy_ids) % 1000 == 0:
        sleep(10)

   # sleep(1)

    t = t1 + timedelta(hours=1/3600)
    t1 = t - timedelta(hours=0.25)

  0%|          | 0/672 [00:00<?, ?it/s]

In [73]:
res.close()
sleep(15)

vacs = []
r = 1
for vac_id in tqdm(set(all_vacancy_ids)):
    if r % 100 == 0:
        sleep(5)
    url = f'https://api.hh.ru/vacancies/{vac_id}'
    res = requests.get(url)
    vacs.append(res.json())
    r +=1 


  0%|          | 0/1508 [00:00<?, ?it/s]